# Aufgabe 1

In [4]:
import numpy as np

# Funktion zum Einlesen der Sequenzen aus Textdateien
def load_sequence(filename):
    with open(filename, 'r') as file:
        return file.read().replace('\n', '')

# Parameter
gap_penalty = -5
match_score = 1
mismatch_penalty = -1

# Lade die Sequenzen
seq1 = load_sequence('Homo_sapiens_hba.txt')
seq2 = load_sequence('Homo_sapiens_mb.txt')

# Initialisiere DP-Matrix und Backtracking-Matrix
dp_matrix = np.zeros((len(seq1) + 1, len(seq2) + 1))
backtrack_matrix = np.zeros((len(seq1) + 1, len(seq2) + 1), dtype=int)

# Fülle die DP-Matrix (lokales Alignment)
max_score = 0
max_pos = (0, 0)

for i in range(1, len(seq1) + 1):
    for j in range(1, len(seq2) + 1):
        match = dp_matrix[i - 1, j - 1] + (match_score if seq1[i - 1] == seq2[j - 1] else mismatch_penalty)
        delete = dp_matrix[i - 1, j] + gap_penalty
        insert = dp_matrix[i, j - 1] + gap_penalty
        dp_matrix[i, j] = max(0, match, delete, insert)
        
        # Speichere den maximalen Score für das Backtracking
        if dp_matrix[i, j] >= max_score:
            max_score = dp_matrix[i, j]
            max_pos = (i, j)

# Backtracking vom höchsten Punkt bis zu einem Wert von 0
aligned_seq1 = []
aligned_seq2 = []
i, j = max_pos

while dp_matrix[i, j] != 0:
    if i > 0 and j > 0 and dp_matrix[i, j] == dp_matrix[i - 1, j - 1] + (match_score if seq1[i - 1] == seq2[j - 1] else mismatch_penalty):
        aligned_seq1.append(seq1[i - 1])
        aligned_seq2.append(seq2[j - 1])
        i -= 1
        j -= 1
    elif i > 0 and dp_matrix[i, j] == dp_matrix[i - 1, j] + gap_penalty:
        aligned_seq1.append(seq1[i - 1])
        aligned_seq2.append('-')
        i -= 1
    else:
        aligned_seq1.append('-')
        aligned_seq2.append(seq2[j - 1])
        j -= 1

# Die Ausrichtung in der korrekten Reihenfolge ausgeben
aligned_seq1 = ''.join(aligned_seq1[::-1])
aligned_seq2 = ''.join(aligned_seq2[::-1])

# Ausgabe
print("Score:", max_score)
print("Alignment:")
print(aligned_seq1)
print(aligned_seq2)

# Speichere das Alignment in einer Datei
with open('local_aln.txt', 'w') as file:
    file.write(f"Score: {max_score}\nAlignment:\n{aligned_seq1}\n{aligned_seq2}\n")


Score: 4.0
Alignment:
TVLT
TVLT


# Aufgabe 2

In [6]:
import numpy as np

# Parameter
gap_open = -10
gap_extend = -0.5
match_score = 1
mismatch_penalty = -1

# Lade die Sequenzen
seq1 = load_sequence('Homo_sapiens_hba.txt')
seq2 = load_sequence('Homo_sapiens_mb.txt')

# Initialisiere Matrizen
S_right = np.full((len(seq1) + 1, len(seq2) + 1), -np.inf)
S_down = np.full((len(seq1) + 1, len(seq2) + 1), -np.inf)
S = np.zeros((len(seq1) + 1, len(seq2) + 1))

# Initialisiere die erste Reihe und Spalte
for i in range(1, len(seq1) + 1):
    S[i, 0] = gap_open + (i - 1) * gap_extend
    S_down[i, 0] = S[i, 0]
for j in range(1, len(seq2) + 1):
    S[0, j] = gap_open + (j - 1) * gap_extend
    S_right[0, j] = S[0, j]

# Fülle die DP-Matrizen
for i in range(1, len(seq1) + 1):
    for j in range(1, len(seq2) + 1):
        S_right[i, j] = max(S_right[i, j - 1] + gap_extend, S[i, j - 1] + gap_open)
        S_down[i, j] = max(S_down[i - 1, j] + gap_extend, S[i - 1, j] + gap_open)
        S[i, j] = max(S[i - 1, j - 1] + (match_score if seq1[i - 1] == seq2[j - 1] else mismatch_penalty), 
                      S_down[i, j], S_right[i, j])

# Berechne den Alignment-Score
alignment_score = S[len(seq1), len(seq2)]

# Ausgabe
print("Score:", alignment_score)
print("S_right Matrix:")
print(S_right)


Score: -89.0
S_right Matrix:
[[ -inf -10.  -10.5 ... -85.5 -86.  -86.5]
 [ -inf -20.   -9.  ... -84.  -84.5 -85. ]
 [ -inf -20.5 -19.  ... -84.5 -85.  -85.5]
 ...
 [ -inf -89.5 -88.  ... -89.  -89.5 -90. ]
 [ -inf -90.  -88.5 ... -87.5 -88.  -88.5]
 [ -inf -90.5 -89.  ... -88.  -88.5 -89. ]]
